In [3]:
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import json
import os

In [4]:
load_dotenv()

True

In [5]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [6]:
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY is not set")

In [7]:
openai = OpenAI()

In [8]:
MODEL = 'gpt-4o-mini'

In [9]:
system_prompt = "You are a gitlab pipeline lookup tool who can easily filters and fetched the gitlab pipelines using the gitlab API"
user_prompt = "Are you a gitlab pipeline lookup tool ?"

In [10]:
message_prompt = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "fetch_gitlab_pipelines",
            "description": "Search for GitLab pipelines using various filters such as task name, status, environment, and date ranges",
            "parameters": {
                "type": "object",
                "properties": {
                    "task_name": {
                        "type": "string",
                        "description": "Name of the task to search for in pipeline names"
                    },
                    "status": {
                        "type": "string",
                        "enum": ["created", "pending", "running", "success", "failed", "canceled"],
                        "description": "Filter pipelines by their current status"
                    },
                    "environment": {
                        "type": "string",
                        "enum": ["development", "test", "uat", "production"],
                        "description": "Filter pipelines by deployment environment"
                    },
                    "updated_after": {
                        "type": "string",
                        "description": "Filter pipelines updated after this date (format: YYYY-MM-DD)"
                    },
                    "updated_before": {
                        "type": "string",
                        "description": "Filter pipelines updated before this date (format: YYYY-MM-DD)"
                    },
                    "username": {
                        "type": "string",
                        "description": "Filter pipelines by the GitLab username who triggered them"
                    }
                },
                "required": ["task_name"]
            }
        }
    }
]

In [12]:
import os
import gitlab


def fetch_gitlab_pipelines(task_name: str, status: str = None, 
                          environment: str = None, updated_after: str = None,
                          updated_before: str = None, username: str = None):
    try:
      project_id = os.getenv('PROJECT_ID')
      gitlab_access_token = os.getenv('GITLAB_ACCESS_TOKEN')
      gitlab_base_url = os.getenv('GITLAB_BASE_URL')

      print("Task name: ", task_name)
      print("Project ID: ", project_id)

      # Configure GitLab client
      gl = gitlab.Gitlab(
          url=gitlab_base_url,
          private_token=gitlab_access_token
      )
      gl.auth()
      project = gl.projects.get(project_id)
    
      # Set up filters
      filters = {
          'per_page': 100,
          'get_all': True
      }

      # Add optional filters
      if status:
          filters['status'] = status
      if environment:
          filters['ref'] = environment
      if username:
          filters['username'] = username

      # Get pipelines with filters
      pipelines = project.pipelines.list(**filters)
      
      output_lines = []
      for pipeline in pipelines:
          pipeline_name = pipeline.name
          if pipeline_name is not None and pipeline_name != '':
              if task_name in pipeline_name:
                  formatted_line = (
                      f"Pipeline ID: {pipeline.id}\n"
                      f"Status: {pipeline.status}\n"
                      f"Environment: {pipeline.ref}\n"
                      f"Created at: {pipeline.created_at}\n"
                      f"Updated at: {pipeline.updated_at}\n"
                      f"Name: {pipeline.name}\n"
                      f"Web URL: {pipeline.web_url}\n"
                      f"{'=' * 50}\n"  # Separator between pipelines
                  )
                  output_lines.append(formatted_line)

      if output_lines:
          final_output = f"Found {len(output_lines)} matching pipelines:\n\n" + \
              "\n".join(output_lines)
          print(f"Found {len(output_lines)} matching pipelines")
          return final_output

      no_results = f"No pipeline found with task name: {task_name}"
      print(no_results)
      return no_results

    except Exception as e:
        error_message = f"Error searching pipelines: {e}"
        print(error_message)
        return error_message
    
    

In [14]:
def chat_with_gpt(message, history):
  messages = history + [{"role": "user", "content": message}]

  completion = openai.chat.completions.create(
      model="gpt-4o",
      messages=messages,
      tools=tools,
  )

  if not completion.choices[0].message.tool_calls:
      return completion.choices[0].message.content

  tool_call = completion.choices[0].message.tool_calls[0]
  print("tool call info: ", tool_call)
  args = json.loads(tool_call.function.arguments)
  result = fetch_gitlab_pipelines(**args)

  messages.append(completion.choices[0].message)
  messages.append({
      "role": "tool",
      "tool_call_id": tool_call.id,
      "content": result
  })

  print("messages: ", messages)

  completion_2 = openai.chat.completions.create(
      model="gpt-4o",
      messages=messages,
      tools=tools,
  )

  return completion_2.choices[0].message.content

In [ ]:
demo = gr.ChatInterface(
    fn=chat_with_gpt,
    type="messages",
    title="Gitlab Bot",
    flagging_mode="never"
)

demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/03/17 12:50:15 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


tool call info:  ChatCompletionMessageToolCall(id='call_EY17XPbwcjl3eWXgpsxnWoFW', function=Function(arguments='{"task_name":"10156526529"}', name='fetch_gitlab_pipelines'), type='function')
Task name:  10156526529
Project ID:  508
Error searching pipelines: 401: invalid_token
messages:  [{'role': 'user', 'metadata': None, 'content': 'fetch pipeline 10156526529', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "It seems like you're trying to fetch details for a specific pipeline using an ID. However, the available tool functions focus on searching for GitLab pipelines based on various filters such as task name, status, environment, and date ranges, rather than fetching by a specific ID.\n\nIf you have additional search criteria or need information about a specific task, please provide more details.", 'options': None}, {'role': 'user', 'content': 'fetch pipeline using task name 10156526529'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=